In [29]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='BvADbqprTah3PYEKGcTHyWxmPkuuaqzBbBVqJbVRAatV',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'hacka-donotdelete-pr-3xr1ws6e2mzpbq'
object_key = 'dataset_processed-1.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']

if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

dataset = pd.read_csv(body)
dataset.head(10)


,label,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,...,pixel_774,pixel_775,pixel_776,pixel_777,pixel_778,pixel_779,pixel_780,pixel_781,pixel_782,pixel_783
0,A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,A,0,0,0,0,0,0,0,0,0,...,93,187,89,0,0,0,0,0,0,0
6,A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
from tensorflow.keras import layers, models

def create_model(input_shape, num_classes):
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))

    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    return model

input_shape = (28, 28, 1)
num_classes = 26           

model = create_model(input_shape, num_classes)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten_1 (Flatten)         (None, 576)              

In [31]:
labels = dataset['label']
pixels = dataset.drop('label', axis=1)

In [32]:
pixels.shape

(17376, 784)

In [33]:
X = pixels.values.reshape(-1, 28, 28, 1)  # Reshape to (num_samples, 28, 28, 1)
X = X / 255.0 

In [34]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

y_encoded = label_encoder.fit_transform(labels)
y_encoded

array([ 0,  0,  0, ..., 25, 25, 25])

In [35]:
y = y_encoded

In [36]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
history = model.fit(
    X_train, y_train,
    epochs=40,
    batch_size=32,
    validation_data=(X_test, y_test)
)

Epoch 1/40
435/435 [==============================] - 8s 17ms/step - loss: 2.6053 - accuracy: 0.2192 - val_loss: 2.1233 - val_accuracy: 0.3527
Epoch 2/40
435/435 [==============================] - 7s 16ms/step - loss: 2.0123 - accuracy: 0.3750 - val_loss: 1.5864 - val_accuracy: 0.5049
Epoch 3/40
435/435 [==============================] - 7s 16ms/step - loss: 1.6233 - accuracy: 0.4924 - val_loss: 1.2776 - val_accuracy: 0.6099
Epoch 4/40
435/435 [==============================] - 7s 17ms/step - loss: 1.3645 - accuracy: 0.5719 - val_loss: 1.2045 - val_accuracy: 0.6162
Epoch 5/40
435/435 [==============================] - 7s 17ms/step - loss: 1.1937 - accuracy: 0.6207 - val_loss: 0.9452 - val_accuracy: 0.7166
Epoch 6/40
435/435 [==============================] - 7s 17ms/step - loss: 1.0459 - accuracy: 0.6654 - val_loss: 0.9094 - val_accuracy: 0.7278
Epoch 7/40
435/435 [==============================] - 7s 16ms/step - loss: 0.9565 - accuracy: 0.6891 - val_loss: 0.8216 - val_accuracy: 0.7503

In [38]:
from ibm_watson_machine_learning import APIClient

wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "pysBFXCh_vbaxZtyGCppB_EkWNL3e95M0x0mLBPHZk2D"
}

client = APIClient(wml_credentials)

In [39]:
space_id = "0d576aa9-fd73-4776-95f6-09aa266318ad"
client.set.default_space(space_id)

'SUCCESS'

In [40]:
model.save('model-2.hd5')

INFO:tensorflow:Assets written to: model-2.hd5/assets


INFO:tensorflow:Assets written to: model-2.hd5/assets


In [41]:
metadata = {
    client.repository.ModelMetaNames.NAME: "Handwriting Recognition Model_2",
    client.repository.ModelMetaNames.TYPE: "tensorflow_2.14", 
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: client.software_specifications.get_uid_by_name("tensorflow_rt24.1-py3.11")
}

model_details = client.repository.store_model(model='model-2.hd5', meta_props=metadata)
print("Model uploaded. Model ID:", model_details["metadata"]["id"])

Model uploaded. Model ID: 7e6db8a6-5d8a-4c2f-ba29-8467270c9fac


In [42]:
deployment_metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Handwriting Recognition Deployment_2",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

deployment_details = client.deployments.create(
    artifact_uid=model_details["metadata"]["id"], 
    meta_props=deployment_metadata
)

print("Deployment created. Deployment ID:", deployment_details["metadata"]["id"])



#######################################################################################

Synchronous deployment creation for uid: '7e6db8a6-5d8a-4c2f-ba29-8467270c9fac' started

#######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
...
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='4fb0eee0-a908-4057-a1d8-a022dbbe1df4'
------------------------------------------------------------------------------------------------


Deployment created. Deployment ID: 4fb0eee0-a908-4057-a1d8-a022dbbe1df4
